## COCO → Yolo 데이터 구조 변환

In [ ]:
import json
import os
from PIL import Image

def convert_bbox(img_width, img_height, bbox):
    x = bbox['x']
    y = bbox['y']
    width = bbox['width']
    height = bbox['height']
    
    x_center = (x + width / 2) / img_width
    y_center = (y + height / 2) / img_height
    width = width / img_width
    height = height / img_height
    
    return x_center, y_center, width, height

def convert_keypoint(img_width, img_height, keypoint):
    x = float(keypoint['x']) / img_width
    y = float(keypoint['y']) / img_height
    return x, y

def convert_json_to_yolo(json_file, img_folder, output_dir):
    try:
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except UnicodeDecodeError:
        # UTF-8로 읽기 실패 시 다른 인코딩 시도
        with open(json_file, 'r', encoding='cp949') as f:
            data = json.load(f)
    
    os.makedirs(output_dir, exist_ok=True)
    
    missing_images = []
    
    for annotation in data['annotations']:
        frame_number = annotation['frame_number']
        timestamp = annotation['timestamp']
        
        img_filename = f"frame_{frame_number}_timestamp_{timestamp}.jpg"
        img_path = os.path.join(img_folder, img_filename)
        
        if not os.path.exists(img_path):
            missing_images.append(img_filename)
            continue
        
        try:
            with Image.open(img_path) as img:
                img_width, img_height = img.size
        except Exception as e:
            print(f"이미지 파일을 열 수 없습니다: {img_path}. 오류: {e}")
            continue
        
        txt_filename = f"frame_{frame_number}_timestamp_{timestamp}.txt"
        txt_path = os.path.join(output_dir, txt_filename)
        
        with open(txt_path, 'w') as f:
            class_id = 0
            
            bbox = annotation['bounding_box']
            x_center, y_center, width, height = convert_bbox(img_width, img_height, bbox)
            
            f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")
            
            keypoints = annotation['keypoints']
            for i in range(1, 16):
                kp = keypoints.get(str(i))
                if kp:
                    x, y = convert_keypoint(img_width, img_height, kp)
                    f.write(f" {x:.6f} {y:.6f}")
                else:
                    f.write(" 0 0")
            
            f.write("\n")
        
        print(f"변환 완료: {txt_path}")
    
    if missing_images:
        print("\n누락된 이미지 파일:")
        for img in missing_images:
            print(img)
        print(f"\n총 {len(missing_images)}개의 이미지 파일이 누락되었습니다.")

# 사용 예
json_file = r"D:\hackerton_6th\dataset\test\cat-arch-010988.json"
img_folder = r"D:\hackerton_6th\dataset\test\cat-arch-010988"
output_dir = r"D:\hackerton_6th\dataset\test\yolo_labels"

convert_json_to_yolo(json_file, img_folder, output_dir)

In [2]:
import cv2
import numpy as np
import os

def draw_label_on_image(img_path, label_path, output_path):
    # 이미지 읽기
    img = cv2.imread(img_path)
    height, width = img.shape[:2]

    # 라벨 파일 읽기
    with open(label_path, 'r') as f:
        label_data = f.read().strip().split()

    # 클래스 ID (여기서는 사용하지 않음)
    class_id = int(label_data[0])

    # 바운딩 박스 정보
    x_center, y_center, box_width, box_height = map(float, label_data[1:5])
    x1 = int((x_center - box_width/2) * width)
    y1 = int((y_center - box_height/2) * height)
    x2 = int((x_center + box_width/2) * width)
    y2 = int((y_center + box_height/2) * height)

    # 바운딩 박스 그리기
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # 키포인트 그리기
    keypoints = list(map(float, label_data[5:]))
    for i in range(0, len(keypoints), 2):
        x = int(keypoints[i] * width)
        y = int(keypoints[i+1] * height)
        cv2.circle(img, (x, y), 3, (0, 0, 255), -1)

    # 결과 이미지 저장
    cv2.imwrite(output_path, img)
    print(f"이미지가 저장되었습니다: {output_path}")

# 사용 예
img_folder = r"D:\hackerton_6th\dataset\test\cat-arch-010988"
label_folder = r"D:\hackerton_6th\dataset\test\yolo_labels"
output_folder = r"D:\hackerton_6th\dataset\test\visualized_labels"

os.makedirs(output_folder, exist_ok=True)

for label_file in os.listdir(label_folder):
    if label_file.endswith('.txt'):
        img_file = label_file.replace('.txt', '.jpg')
        img_path = os.path.join(img_folder, img_file)
        label_path = os.path.join(label_folder, label_file)
        output_path = os.path.join(output_folder, f"visualized_{img_file}")

        if os.path.exists(img_path):
            draw_label_on_image(img_path, label_path, output_path)
        else:
            print(f"이미지 파일을 찾을 수 없습니다: {img_path}")

print("모든 이미지 처리가 완료되었습니다.")

이미지가 저장되었습니다: D:\hackerton_6th\dataset\test\visualized_labels\visualized_frame_0_timestamp_0.jpg
이미지가 저장되었습니다: D:\hackerton_6th\dataset\test\visualized_labels\visualized_frame_100_timestamp_4000.jpg
이미지가 저장되었습니다: D:\hackerton_6th\dataset\test\visualized_labels\visualized_frame_105_timestamp_4200.jpg
이미지가 저장되었습니다: D:\hackerton_6th\dataset\test\visualized_labels\visualized_frame_10_timestamp_400.jpg
이미지가 저장되었습니다: D:\hackerton_6th\dataset\test\visualized_labels\visualized_frame_110_timestamp_4400.jpg
이미지가 저장되었습니다: D:\hackerton_6th\dataset\test\visualized_labels\visualized_frame_115_timestamp_4600.jpg
이미지가 저장되었습니다: D:\hackerton_6th\dataset\test\visualized_labels\visualized_frame_120_timestamp_4800.jpg
이미지가 저장되었습니다: D:\hackerton_6th\dataset\test\visualized_labels\visualized_frame_125_timestamp_5000.jpg
이미지가 저장되었습니다: D:\hackerton_6th\dataset\test\visualized_labels\visualized_frame_130_timestamp_5200.jpg
이미지가 저장되었습니다: D:\hackerton_6th\dataset\test\visualized_labels\visualized_frame_135_timest